In [1]:
import sys
sys.path.append('../src/')

import config
import logging
import scanpy as sc
from refcm import RefCM

config.start_logging(logging.DEBUG)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
ds = ['MTG', 'ALM', 'VISp']
ds = {s: sc.read_h5ad(f'../data/{s}.h5ad') for s in ds}

[h5py._conv      ] [DEBUG   ] : Creating converter from 3 to 5


In [4]:
ds['MTG'].to_df()

,X0610007P14Rik,X0610009O20Rik,X0610010F05Rik,X0610011F06Rik,X0610030E20Rik,X0610037L13Rik,X0610040J01Rik,X1110001J03Rik,X1110004E09Rik,X1110004F10Rik,...,Zwint,Zxdb,Zxdc,Zyg11a,Zyg11b,Zyx,Zzef1,Zzz3,a,l7Rn6
F1S4_160106_001_B01,0.0,157.0,9.0,12.0,0.0,0.0,88.0,1.0,268.0,258.0,...,0.0,0.0,0.0,0.0,33.0,0.0,1.0,306.0,0.0,374.0
F1S4_160106_001_C01,107.0,80.0,85.0,0.0,0.0,245.0,1.0,0.0,327.0,521.0,...,0.0,17.0,35.0,0.0,258.0,0.0,8.0,341.0,0.0,189.0
F1S4_160106_001_E01,53.0,36.0,19.0,12.0,0.0,23.0,0.0,0.0,196.0,96.0,...,0.0,0.0,13.0,25.0,45.0,72.0,197.0,296.0,2.0,33.0
F1S4_160106_001_G01,2.0,1.0,344.0,31.0,0.0,82.0,0.0,36.0,45.0,66.0,...,0.0,0.0,82.0,0.0,181.0,0.0,58.0,206.0,28.0,97.0
F1S4_160106_001_H01,97.0,0.0,264.0,103.0,37.0,0.0,1.0,0.0,40.0,122.0,...,0.0,0.0,0.0,0.0,317.0,10.0,169.0,0.0,12.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
F2S4_170405_060_A01,46.0,34.0,232.0,51.0,0.0,41.0,0.0,0.0,0.0,209.0,...,0.0,0.0,49.0,0.0,160.0,51.0,46.0,352.0,0.0,76.0
F2S4_170405_060_B01,0.0,186.0,156.0,29.0,0.0,0.0,0.0,0.0,54.0,202.0,...,75.0,0.0,87.0,0.0,128.0,72.0,22.0,370.0,1.0,69.0
F2S4_170405_060_C01,90.0,237.0,456.0,98.0,28.0,0.0,0.0,0.0,1.0,127.0,...,115.0,0.0,171.0,1.0,79.0,23.0,0.0,113.0,26.0,0.0
F2S4_170405_060_E01,0.0,0.0,400.0,0.0,0.0,92.0,94.0,0.0,0.0,75.0,...,0.0,0.0,141.0,0.0,0.0,62.0,316.0,187.0,0.0,0.0


In [5]:
ds['ALM'].to_df()

,X0610007P14Rik,X0610009O20Rik,X0610010F05Rik,X0610011F06Rik,X0610030E20Rik,X0610037L13Rik,X0610040J01Rik,X1110001J03Rik,X1110004E09Rik,X1110004F10Rik,...,Zwint,Zxdb,Zxdc,Zyg11a,Zyg11b,Zyx,Zzef1,Zzz3,a,l7Rn6
F1S4_180124_314_A01,80.0,11.0,135.0,32.0,0.0,31.0,1.0,49.0,22.0,204.0,...,3190.0,0.0,17.0,0.0,96.0,0.0,0.0,45.0,0.0,73.0
F1S4_180124_315_A01,233.0,37.0,183.0,114.0,0.0,189.0,0.0,244.0,23.0,484.0,...,4832.0,28.0,33.0,0.0,179.0,0.0,0.0,83.0,0.0,127.0
F1S4_180124_315_B01,142.0,4.0,129.0,9.0,14.0,73.0,0.0,196.0,57.0,479.0,...,3487.0,29.0,4.0,0.0,124.0,33.0,37.0,21.0,0.0,93.0
F1S4_180124_315_C01,54.0,1.0,28.0,40.0,17.0,67.0,0.0,55.0,36.0,388.0,...,3891.0,13.0,0.0,0.0,98.0,0.0,5.0,18.0,0.0,135.0
F1S4_180124_315_D01,139.0,0.0,22.0,117.0,10.0,105.0,0.0,91.0,70.0,579.0,...,4537.0,0.0,0.0,0.0,93.0,26.0,22.0,29.0,0.0,81.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FJS4_170511_008_A01,196.0,1.0,140.0,162.0,14.0,146.0,0.0,178.0,125.0,447.0,...,3313.0,0.0,4.0,0.0,349.0,8.0,40.0,129.0,7.0,210.0
FJS4_170511_008_D01,122.0,17.0,288.0,120.0,126.0,113.0,0.0,49.0,146.0,268.0,...,1322.0,0.0,47.0,0.0,418.0,20.0,62.0,0.0,0.0,222.0
FJS4_170511_008_G01,196.0,2.0,273.0,99.0,24.0,242.0,0.0,194.0,69.0,612.0,...,2957.0,49.0,60.0,0.0,527.0,24.0,123.0,75.0,0.0,485.0
FJS4_170511_008_H01,353.0,25.0,326.0,1.0,0.0,42.0,21.0,122.0,76.0,362.0,...,2992.0,8.0,0.0,0.0,468.0,0.0,91.0,46.0,0.0,166.0


In [7]:
q, ref = 'MTG', 'ALM'

rcm = RefCM(cache_load=False, cache_save=False, discovery_threshold=None)
m = rcm.annotate(ds[q], q, ds[ref], ref, 'labels34', 'labels34')
m.eval('labels34')
m.display_matching_costs('labels34')

[refcm           ] [INFO    ] : NOTE: raw counts expected in anndata .X attributes.
[refcm           ] [DEBUG   ] : Loading cached mapping costs from cache.json.
[embeddings      ] [DEBUG   ] : Using 1948 genes.
[refcm           ] [DEBUG   ] : Computing Wasserstein distances.
|████████████████| [100.00% ] : 00:30
[refcm           ] [DEBUG   ] : starting LP optimization
[refcm           ] [DEBUG   ] : optimization terminated w. status "Optimal"
[matchings       ] [DEBUG   ] : [+|1.00] Astrocyte            mapped to Astrocyte           
[matchings       ] [DEBUG   ] : [+|1.00] Chandelier           mapped to Chandelier          
[matchings       ] [DEBUG   ] : [+|1.00] Exc L2/3 IT          mapped to Exc L2/3 IT         
[matchings       ] [DEBUG   ] : [+|1.00] Exc L3/5 IT          mapped to Exc L3/5 IT         
[matchings       ] [DEBUG   ] : [+|1.00] Exc L4/5 IT          mapped to Exc L4/5 IT         
[matchings       ] [DEBUG   ] : [+|1.00] Exc L5 PT            mapped to Exc L5 PT      

In [15]:
m.display_matching_costs('labels34', show_all_labels=True, width=800, height=800)

In [5]:
ds[q].obs

,labels3,labels34,refcm_clusters,refcm_annot
F1S4_160106_001_B01,Inhibitory,Sst 1,22,Sst 1
F1S4_160106_001_C01,Excitatory,Exc L5/6 IT 3,8,Exc L5/6 IT 3
F1S4_160106_001_E01,Excitatory,Exc L6 CT,10,Exc L6 CT
F1S4_160106_001_G01,Excitatory,Exc L6b,13,Exc L6b
F1S4_160106_001_H01,Excitatory,Exc L4/5 IT,4,Exc L4/5 IT
...,...,...,...,...
F2S4_170405_060_A01,Excitatory,Exc L6 IT 1,11,Exc L6 IT 1
F2S4_170405_060_B01,Excitatory,Exc L6 CT,10,Exc L6 CT
F2S4_170405_060_C01,Excitatory,Exc L6 IT 2,12,Exc L6 IT 2
F2S4_170405_060_E01,Excitatory,Exc L5/6 NP,9,Exc L5/6 NP


In [23]:
x_qc = ds[q][ds[q].obs['labels34'] == 'Sst 1'].X
x_rc = ds[ref][ds[ref].obs['labels34'] == 'Sst 1'].X

In [73]:
pairwise_distances(x_qc, x_rc, 'minkowski')

array([[59909.19050743, 56357.54638598, 52861.26636036, ...,
        61214.78589675, 62135.52360843, 62075.6845127 ],
       [67211.92684129, 63463.76467347, 60927.44072739, ...,
        67983.56214984, 68538.19934368, 68490.28972909],
       [90120.8554905 , 85935.18254654, 82238.57715996, ...,
        91429.04553188, 92395.87295139, 92111.88221149],
       ...,
       [53138.74682112, 51214.56015132, 47497.87477183, ...,
        56756.65536079, 57870.05302754, 57168.47178404],
       [62435.16385963, 60938.99204681, 57158.60263075, ...,
        65852.23984812, 66471.62188793, 65906.88750941],
       [87827.94473116, 83447.58410701, 79276.44160075, ...,
        87818.80049358, 88217.90147435, 88358.94445056]])

In [70]:
(x_qc @ x_rc.T) / (norm(x_qc, axis=1).reshape(-1, 1) @ norm(x_rc, axis=1).reshape(1, -1))

array([[0.27448392, 0.4381256 , 0.5240184 , ..., 0.32948738, 0.31564957,
        0.30325562],
       [0.2888411 , 0.4324723 , 0.495603  , ..., 0.3288856 , 0.32230005,
        0.31314394],
       [0.13322608, 0.31570604, 0.412918  , ..., 0.18174323, 0.16464649,
        0.16035235],
       ...,
       [0.2333373 , 0.39687553, 0.49045992, ..., 0.27253112, 0.2565909 ,
        0.2553947 ],
       [0.18037313, 0.324394  , 0.42278203, ..., 0.21361275, 0.20980869,
        0.20703687],
       [0.22946192, 0.3800401 , 0.48740515, ..., 0.273969  , 0.269046  ,
        0.25815186]], dtype=float32)

In [33]:
q, ref = 'MTG', 'ALM'

rcm = RefCM(cache_load=False, cache_save=False, discovery_threshold=None)
m = rcm.annotate(ds[q], q, ds[ref], ref, 'labels34', 'labels34')
m.eval('labels34')
m.display_matching_costs('labels34')

[refcm           ] [INFO    ] : NOTE: raw counts expected in anndata .X attributes.
[refcm           ] [DEBUG   ] : Loading cached mapping costs from cache.json.


In [6]:
rcm = RefCM(discovery_threshold=0)

for i, q_id in enumerate(ds):
    for ref_id in [d for d in ds if d != q_id]:
        m = rcm.annotate(ds[q_id], q_id, ds[ref_id], ref_id, 'labels34', 'labels34')
        m.eval('labels34')

[refcm           ] [INFO    ] : NOTE: raw counts expected in anndata .X attributes.
|████████████████| [100.00% ] : 00:38
[matchings       ] [INFO    ] : mapped MTG                  to ALM                 
[matchings       ] [INFO    ] : (34 common cell types)
[matchings       ] [INFO    ] : 34/34 correct mappings
[matchings       ] [INFO    ] : 0 /34 incorrect mappings
|████████████████| [100.00% ] : 00:43
[matchings       ] [INFO    ] : mapped MTG                  to VISp                
[matchings       ] [INFO    ] : (34 common cell types)
[matchings       ] [INFO    ] : 34/34 correct mappings
[matchings       ] [INFO    ] : 0 /34 incorrect mappings
|████████████████| [100.00% ] : 00:36
[matchings       ] [INFO    ] : mapped ALM                  to MTG                 
[matchings       ] [INFO    ] : (34 common cell types)
[matchings       ] [INFO    ] : 34/34 correct mappings
[matchings       ] [INFO    ] : 0 /34 incorrect mappings
|████████████████| [100.00% ] : 01:01
[matchings 